In [13]:
include("MiniCollectiveSpins.jl")
using PyPlot
using Statistics
using JLD2
using OrdinaryDiffEq
import PhysicalConstants.CODATA2018: c_0
using Unitful
using ProgressMeter
using NonlinearSolve
using SteadyStateDiffEq 
using BenchmarkTools

In [ ]:
""" Prepare the initial vector u0 """
function u0_CFunction(phi_array, theta_array, op_list)
    u0 = ones(ComplexF64, length(op_list))
    for i in 1:length(op_list)
        if length(op_list[i]) == 1
            j = Int(op_list[i][1] % 10^floor(log10(abs(op_list[i][1]))-1)) # Atom nbr
            if parse(Int, string(op_list[i][1])[1:2]) == 22
                u0[i] = cos(theta_array[j]/2)^2
            elseif parse(Int, string(op_list[i][1])[1:2]) == 21
                u0[i] = cos(theta_array[j]/2)*exp(1im*phi_array[j])*sin(theta_array[j]/2)
            else
                println(op_list[i][1])
            end
        end

        if length(op_list[i]) == 2
            for op in op_list[i]
                j = Int(op % 10^floor(log10(abs(op))-1)) # Atom nbr
                if parse(Int, string(op)[1:2]) == 22
                    u0[i] *= cos(theta_array[j]/2)^2
                elseif parse(Int, string(op)[1:2]) == 21
                    u0[i] *= cos(theta_array[j]/2)*exp(1im*phi_array[j])*sin(theta_array[j]/2)
                elseif parse(Int, string(op)[1:2]) == 12
                    u0[i] *= cos(theta_array[j]/2)*exp(-1im*phi_array[j])*sin(theta_array[j]/2)
                else
                    println(op)
                end
            end
        end
    end
    return u0
end


""" Create a random distribution, save it, computes the corresponding parameters an return the stationnary state. 
If compute_t_evolution, compute the whole evolution, else only the stationnary state. """
function solve_random_distrib(chunk, f, op_list, N, n, d0_lb, compute_t_evolution)
    popup_ss, nbr_error_ss = [], []
    popup_t, nbr_error_t = [], []

    for i in chunk
        # Compute distribution
        L = (N/n)^(1/3) # Change the volume to keep the density cste
        r0 = [[rand(Float64)*L, rand(Float64)*L, rand(Float64)*L] for i in 1:N]

        # Choose a distribution where the minimum distance between the atoms is bigger than d0_min
        while min_r0(r0) < d0_lb
            r0 = [[rand(Float64)*L, rand(Float64)*L, rand(Float64)*L] for i in 1:N]
        end

        # Save the atoms position for comparison with QuantumOptics
        @save "r0/r0_N_$(N)_r_$i.jdl2" r0 L

        # Compute the parameters
        system = SpinCollection(r0, e, gammas=1.)
        Ω_CS = OmegaMatrix(system)
        Γ_CS = GammaMatrix(system)
        Γij_ = [Γ_CS[i, j] for i = 1:N for j=1:N]
        Ωij_ = [Ω_CS[i, j] for i = 1:N for j=1:N if i≠j]
        exp_RO_ = [exp(1im*r0[i]'kl) for i = N:-1:1] # We go in the decreasing direction to avoid exp_RO(10) being replace by exp_RO(1)0
        conj_exp_RO_ = [exp(-1im*r0[i]'kl) for i = N:-1:1]
        p0 = ComplexF64.([Γij_; Ωij_; exp_RO_; conj_exp_RO_; Ω_RO/2])
        
        # Load the functions
        fsolve(du, u, p, t) = f(du, u, p0)

        if compute_t_evolution
            phi_array_0, theta_array_0 = zeros(N), ones(N)*π # We start from all the atoms in the GS
            u0 = u0_CFunction(phi_array_0, theta_array_0, op_list)

            prob = OrdinaryDiffEq.ODEProblem(fsolve, u0, (T[1], T[end]))

            sol = OrdinaryDiffEq.solve(prob, OrdinaryDiffEq.DP5();
                        reltol=1.0e-5,
                        abstol=1.0e-5) # , saveat=T

            if SciMLBase.successful_retcode(sol)
                push!(popup_t, [sum(real(sol.u[i][1:N])) for i=1:length(T)])
            else
                push!(nbr_error_t, i)
            end
            uf = sol.u[end]

        else
            phi_array_f, theta_array_f = zeros(N), ones(N)*3*π/4 # We start from all the atoms in the GS
            uf = u0_CFunction(phi_array_f, theta_array_f, op_list)
        end

        ## Computation steady state ##
        prob_ss = SteadyStateProblem(fsolve, uf)
        sol_ss = solve(prob_ss, DynamicSS(DP5()); abstol=1e-1, reltol=1e-1) # ; maxiters=10_000, abstol=1e-1, reltol=1e-1
        if SciMLBase.successful_retcode(sol_ss)
                push!(popup_ss, sum(real(sol_ss[1:N])))
            else
                push!(nbr_error_ss, i)
        end

    end
    return popup_ss, nbr_error_ss, popup_t, nbr_error_t 
end

""" Return the minimum distance of a distribution of atoms r0 """
function min_r0(r0)
    N = length(r0)
    d0 = zeros(N, N) # Repetiton, atom i, distance from atom j
    for j in 1:N
        for k = 1:N
            d0[j, k] = norm(r0[j]-r0[k])
        end
    end
    return minimum(d0[d0 .> 0])
end

min_r0

### Define the system

In [15]:
# Nbr of particles
N = 4
r = 1 # Nbr of repetitions

# Normalisation parameters
λ = 421e-9
γ = 32.7e6 # In Hz

# Physical values
ω0 = (2π*ustrip(c_0)/λ)
ωl = ω0
kl = [ustrip(c_0)/ωl, 0, 0] # Laser along x
Ω_RO = 1e7 # Taken from Barbut arXiv:2412.02541v1

# Fixed density
n0 = 2e3 # atoms per unit of volume (already normalized)
d0_lb = 1e-10 # Minimum distance between the atoms (lower boundary) in m

# Normalization
ω0 = ω0 / γ
ωl = ωl / γ
kl = kl * λ
Ω_RO = Ω_RO / γ
d0_lb = d0_lb/λ

# Quantization axis along z
e = [0, 0, 1.]

# Integration parameter
tstep = 0.1
T = [0:tstep:100;]; # Normalised time

### Compute stationnary state for r repetitions

In [16]:
# Create the directories
if !isdir("r0")
    mkdir("r0")
end
if !isdir("Images_distribution")
    mkdir("Images_distribution")
end
if !isdir("solutions")
    mkdir("solutions")
end
nothing 

In [17]:
# Prepare the function
libpath ="libs/liballfuncs_$N.dll"
f(du, u, params) = ccall(("diffeqf", libpath), Cvoid, (Ptr{ComplexF64}, Ptr{ComplexF64}, Ptr{ComplexF64}), du, u, params)

f (generic function with 1 method)

In [18]:
@load "op_list/op_list_$N.jdl2" op_list
list_r = 1:r
chunks = Iterators.partition(list_r, cld(length(list_r), Threads.nthreads()))

solve_random_distrib(1, f, op_list, N, n0, d0_lb, true)

tasks = map(chunks) do chunk # Split the different distributions into chuncks solved on each core
    Threads.@spawn solve_random_distrib(chunk, f, op_list, N, n0, d0_lb, true)
end

# Gather the data from the different threads
sol_tasks = fetch.(tasks)
popup_ss, nbr_error_ss = vcat([s[1] for s in sol_tasks]...), vcat([s[2] for s in sol_tasks]...)
popup_t, nbr_error_t = vcat([s[3] for s in sol_tasks]...), vcat([s[4] for s in sol_tasks]...)
println("Nbr errors SS = "*string(length(nbr_error_ss)))
println("Nbr errors t_evol = "*string(length(nbr_error_t)))

┌ Warning: At t=0.8935477101079715, dt was forced below floating point epsilon 1.1102230246251565e-16, and step error estimate = 1.6512704830347036. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of ComplexF64).
└ @ SciMLBase C:\Users\Joschka\.julia\packages\SciMLBase\TZ9Rx\src\integrator_interface.jl:671
┌ Warning: At t=7.343259700316642e-16, dt was forced below floating point epsilon 9.860761315262648e-32, and step error estimate = 2.7968908283671543. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of ComplexF64).
└ @ SciMLBase C:\Users\Joschka\.julia\packages\SciMLBase\TZ9Rx\src\integrator_interface.jl:671
┌ Warning: At t=2.1987481422932706, dt was forced below floating point epsilon 4.440892098500626e-16, and step error estimate = 2.0112300803867695. Aborting. Ther

Nbr errors SS = 1
Nbr errors t_evol = 1


In [19]:
@save "solutions/sol_N_$(N)_r_$(r)" popup_ss nbr_error_ss popup_t nbr_error_t

In [20]:
@load "solutions/sol_N_$(N)_r_$(r)" popup_ss nbr_error_ss popup_t nbr_error_t

4-element Vector{Symbol}:
 :popup_ss
 :nbr_error_ss
 :popup_t
 :nbr_error_t

# Plots SS

In [21]:
fig, ax = subplots()
for i in 1:length(popup_ss)
    ax.hlines(popup_ss[i], T[1], T[end], linestyle="--", color = line.get_color())
end
ax.set_xlabel(L"$\gamma t$")
ax.set_ylabel(L"$\langle  n_{\uparrow} \rangle $")

suptitle("N = $N, r = $r, Starting from "*L"$|\downarrow \downarrow \rangle $")
pygui(false);
# pygui(true); show()

false

# Plots with time evolution

In [22]:
fig, ax = subplots()
for i in 1:length(popup_t)
    line, = ax.plot(T, popup_t[i])
    # ax.hlines(popup_ss[i], T[1], T[end], linestyle="--", color = line.get_color())
end
ax.set_xlabel(L"$\gamma t$")
ax.set_ylabel(L"$\langle  n_{\uparrow} \rangle $")

suptitle("N = $N, r = $r, Starting from "*L"$|\downarrow \downarrow \rangle $")
pygui(false);
# pygui(true); show()

false